# Quel constructeur a le plus de casse dans sa voiture, et quels sont les problèmes les plus courants ?

In [102]:
import pandas as pd

status_code  = pd.read_csv('../data/status.csv')
f1_constructor = pd.read_csv('../data/constructors.csv')
# f1_c_standings = pd.read_csv('../data/constructor_standings.csv')
# races = pd.read_csv('../data/races.csv')
results = pd.read_csv('../data/results.csv')
# f1_constructor_results = pd.read_csv('../data/constructor_results.csv')

## Construction de la table

On a besoin des informations à propos des courses pour savoir comment elles se sont terminé. Donc, on lie les tables `f1_constructor`et `results`. Ensuite, on les liera avec `status_code` pour récupérer les status codes.

In [103]:
results_x_constructor = pd.merge(results, f1_constructor, on='constructorId')[["constructorId", "statusId"]]

# On compte les occurrences de chaque message d'erreur pour chaque constructeur
status_counts = results_x_constructor.groupby(['constructorId', 'statusId']).size().reset_index(name='count')

# On enlève les statusId qui ne sont pas liés à un défaut de voiture (par exemple "course réussi", ou "pilote malade")
status_counts = status_counts[
    ~status_counts['statusId'].isin([1, 2, 3, 4, 31, 50, 128, 53, 58, 73, 81, 82, 88, 97, 100] + list(range(11, 20)))]

# Calculer le total des occurrences de résultats par constructeur (le nombre de courses auxquels ils ont participé)
total_counts = results_x_constructor.groupby('constructorId').size().reset_index(name='total_count')

# Ajouter la colonne de proportion de problèmes dans le tableau
status_counts = pd.merge(status_counts, total_counts, on='constructorId')
status_counts['proportion'] = status_counts['count'] / status_counts['total_count']

## Problèmes les plus fréquents par constructeur

In [104]:

# Trie par 'constructorId' et 'count' (desc), et sélection des 10 principaux status ID par constructeur
status_counts_per_constructor = status_counts.sort_values(by=['constructorId', 'count'], ascending=[True, False])
status_counts_per_constructor = status_counts_per_constructor.groupby('constructorId').head(10)

# Fusion avec la table `status_code` et inclut les noms des constructeurs
status_counts_per_constructor = pd.merge(status_counts_per_constructor, status_code, on='statusId')
status_counts_per_constructor = pd.merge(status_counts_per_constructor, f1_constructor[['constructorId', 'name']], on='constructorId')

# status_counts_per_constructor.drop(["statusId","constructorId"], axis=1, inplace=True)
status_counts_per_constructor.head()

,constructorId,statusId,count,total_count,proportion,status,name
0,1,5,126,1899,0.066351,Engine,McLaren
1,1,20,46,1899,0.024223,Spun off,McLaren
2,1,6,38,1899,0.020011,Gearbox,McLaren
3,1,10,29,1899,0.015271,Electrical,McLaren
4,1,22,17,1899,0.008952,Suspension,McLaren


## Constructeur avec le plus de problèmes (max occurrences)

On regarde quel constructeur rencontre le plus d'erreur.

In [105]:
status_counts_constructor = status_counts_per_constructor.groupby('name')[['count','constructorId']].sum().reset_index()

status_counts_constructor.sort_values(by='count', ascending=False)

,name,count,constructorId
64,Ferrari,397,60
116,McLaren,307,10
177,Tyrrell,262,250
166,Team Lotus,262,320
124,Minardi,248,180
...,...,...,...
156,Snowberger,1,162
167,Tec-Mec,1,124
170,Token,1,79
176,Turner,1,142


### Passage en proportion

Même si *Ferrari* et *McLaren* ont de loin le plus de panne, c'est aussi parce que ce sont les constructeurs les plus producteurs. Regardons plutôt la proportion de pannes par course.

In [106]:
# Calcul de la proportion du nombre de pannes par course de chaque constructeur
failures_per_constructor = status_counts_constructor.merge(total_counts, on='constructorId')
failures_per_constructor['failure_proportion'] = failures_per_constructor['count'] / failures_per_constructor[
    'total_count']

# Tri des résultats en ordre décroissant par proportion de pannes
failures_per_constructor_sorted = failures_per_constructor.sort_values(by='failure_proportion', ascending=False)

failures_per_constructor_sorted[['name', 'failure_proportion']]

,name,failure_proportion
6,BAR,72.000000
25,Force India,36.000000
51,Red Bull,31.000000
53,Sauber,17.125000
24,Ferrari,12.406250
...,...,...
58,Stebro,0.050000
57,Spyker MF1,0.037037
12,Brawn,0.029412
8,BMW Sauber,0.020833
